In [1]:
import pandas as pd


In [2]:
excel_path = "../data/4_GEOMAR-relevant_COG_grantees_and_panel_members_2013-2024.xlsx"
sheet_name = "Panel members"
filter_call = "CoG 2024"

In [3]:
df = pd.read_excel(excel_path, sheet_name=sheet_name)
# strip the column names
df.columns = df.columns.str.strip().str.replace(r'\s+', ' ', regex=True)
display(df.head())

# filter
#df = df[df['Call'] == filter_call].reset_index(drop=True)

# Zeige alle Spaltennamen
print("Verfügbare Spalten:")
print(df.columns.tolist())



# Nur wenn die Spalten existieren
if 'First name' in df.columns and 'Last name' in df.columns:
    df['Name'] = df["First name"] + " " + df["Last name"].str.strip()
else:
    print("\n⚠️ Kann 'Name' nicht erstellen - Spalten fehlen!")

display(df.head())

,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call
0,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Member,LS,CoG 2014
1,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Chair,LS,CoG 2016
2,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2014
3,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2016
4,Ameli,Nadia,UCL Institute for Sustainable Resources,"Climate Finance. climate finance, networks and...",SH7,Member,SH,CoG 2024


Verfügbare Spalten:
['Last name', 'First name', 'Host institution (HI)', 'Field', 'Panel', 'Function', 'Domain', 'Call']


,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call,Name
0,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Member,LS,CoG 2014,Jon Agren
1,Agren,Jon,University of Uppsala,Ecology and evolution of plant populations; Ev...,LS8,Chair,LS,CoG 2016,Jon Agren
2,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2014,Francis Albarede
3,Albarede,Francis,ENS Lyon,"geochemistry; Trace elements, radiogenic isoto...",PE10,Member,PE,CoG 2016,Francis Albarede
4,Ameli,Nadia,UCL Institute for Sustainable Resources,"Climate Finance. climate finance, networks and...",SH7,Member,SH,CoG 2024,Nadia Ameli


In [4]:
# show dataframe where Name is not null and other columnas contain Nan
df_member = df[df['Name'].notnull()].reset_index(drop=True)

empty_fields = df_member[df_member["Field"].isnull()]
display(empty_fields)

unique_members = empty_fields['Name'].unique()
print(f"\nAnzahl der einzigartigen Mitglieder ohne 'Field': {len(unique_members)}")



,Last name,First name,Host institution (HI),Field,Panel,Function,Domain,Call,Name
8,Anderluh,Gregor,NaN,NaN,LS9,Member,LS,CoG 2021,Gregor Anderluh
9,Anderluh,Gregor,NaN,NaN,LS9,Member,LS,CoG 2023,Gregor Anderluh
15,Arndt,Nicholas,NaN,NaN,PE10,Member,PE,CoG 2021,Nicholas Arndt
16,Arndt,Nicholas,NaN,NaN,PE10,Member,PE,CoG 2023,Nicholas Arndt
19,Bai,Yuling,NaN,NaN,LS9,Member,LS,CoG 2021,Yuling Bai
...,...,...,...,...,...,...,...,...,...
543,Virrantaus,Kirsi,NaN,NaN,SH7,Member,SH,CoG 2021,Kirsi Virrantaus
544,Virrantaus,Kirsi,NaN,NaN,SH7,Member,SH,CoG 2023,Kirsi Virrantaus
561,Wu,Xiaogang,NaN,NaN,SH7,Member,SH,CoG 2021,Xiaogang Wu
562,Xu,Chong-Yu,NaN,NaN,PE10,Member,PE,CoG 2021,Chong-Yu Xu



Anzahl der einzigartigen Mitglieder ohne 'Field': 68


In [5]:
from pprint import pprint

pprint(unique_members)
pprint(len(unique_members))

array(['Gregor Anderluh', 'Nicholas Arndt', 'Yuling Bai',
       'Michael Benton', 'Petra Boevink', 'Peter Bols',
       'Kim A. Boutilier', 'Donald E. Canfield', 'Jørgen Carling',
       'Paul Cloke', 'Marcus Collier', 'Pierre Phillippe Combes',
       'Pierre-Philippe Combes', 'Cláudia de Freitas', 'Ben Derudder',
       'Paul Ekins', 'Martin Funk', 'Toni Gabaldón', 'Andrey Ganopolski',
       'Alexandros Gasparatos', 'Ashleigh Griffin', 'Lars-Anders Hansson',
       'Marie E. Herberstein', 'Niels Høiby', 'Robert Holt',
       'Corinna Hoose', 'Dirk Jacobs', 'Adrian Jenkins', 'Nadja Kabisch',
       'Henrik Kaessmann', 'Inger Kappel Schmidt', 'Kristine Kern',
       'Phoebe Koundouri', 'Julie LaRoche', 'Iñigo Lasa Uzcudun',
       'Philippe Lemanceau', 'Edward Lemke', 'Regina Lindborg',
       'Rosaly Lopes', 'Alessandra Magistrato', 'Johanna Mappes',
       'Anders Meibom', 'Andrzej Mizgajski', 'Andrew Mondschein',
       'Michael Obersteiner', 'Spyros Pandis', 'Dario Papale',
     

In [6]:
import ollama 
import time
import requests

def fetch_openalex_id(name:str) -> dict | None:

    search_name = name.replace(" ", "+")
    base_url = f"https://api.openalex.org/authors?filter=display_name.search:{search_name}"
    response = requests.get(base_url)
    if response.status_code == 200:
        data = response.json()
        results_list = data.get('results', [])
        
        if not results_list:
            print(f"⚠️ Keine Ergebnisse für {name}")
            return None
        
        # Nimm das erste Ergebnis
        first_result = results_list[0]
        #pprint(type(first_result))
        #pprint(first_result.keys())
        
        # Extrahiere Informationen
        topics = first_result.get('topics', [])

        # 🔹 Sortieren nach 'count' (absteigend)
        sorted_topics = sorted(topics, key=lambda x: x['count'], reverse=True)

        # 🔹 Die größten 4 auswählen
        top_topics = [t['display_name'] for t in sorted_topics[:1]]

        #pprint(top_topics)
        affiliations = first_result.get('affiliations', [])
        x_concepts = first_result.get('x_concepts', [])

        topics_fields_list = [topic['display_name'] for topic in topics]
        #pprint(topics_fields_list)
        
        affiliation = affiliations[0]["institution"]["display_name"] if affiliations else None
        #pprint(affiliation)
        
        x_concepts_fields_list = [concept['display_name'] for concept in x_concepts if concept["score"] >= 90]
        #pprint(x_concepts_fields_list)
        #pprint(x_concepts)
        

        # Speichere die Ergebnisse in einem Dictionary
        results = {
            "Name": name,
            "topics": top_topics,
            "affiliation": affiliation,
            "x_concepts": x_concepts_fields_list
        }
        return results
    else:
        print(f"⚠️ Fehler bei der Anfrage: {response.status_code}")
        return None
    


def process_researcher_profile(results:dict, stream:bool = False, model:str = "deepseek-r1:1.5b") -> str:

    prompt1 = f"""
    You are a scientific summarizer. Based on the following structured information about a researcher, write a concise, natural-sounding academic profile of about 3 sentences. Focus on their main research areas, using fluent academic language similar to a bio in a research paper.
    Do not call them postdoc or similar terms; just focus on their expertise and research themes.
    Input data:
    
    {results}

    Output format:
    A short paragraph (≈3 sentences) describing the person’s expertise and research themes, written in English.
    """

    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt1}],
        stream=True,
    )

    # Antwort puffern und gleichzeitig anzeigen
    response_text = ""
    for chunk in response:
        content = chunk.get("message", {}).get("content", "")
        print(content, end="", flush=True)
        response_text += content

    # # Jetzt hast du alles gesammelt
    # print("\n\n✅ Streaming beendet.")

    return response_text


df_member_info = pd.DataFrame(columns=["name", "affiliation", "topics", "x_concepts", "generated_profile"])

for member in unique_members[:]:
    results = fetch_openalex_id(member)
    if results:
        print(f"\n\n--- Profil für {member} ---")
        response_text = process_researcher_profile(
            model="deepseek-r1:1.5b",
            results=results,
            stream=True
        )
        #print(f"\nGeneriertes Profil:\n{response_text}")
        time.sleep(2)  # Kurze Pause zwischen den Anfragen
    else:
        print(f"Keine Daten für {member} gefunden.")

    df_member_info = pd.concat([df_member_info, pd.DataFrame([{
        "name": member,
        "affiliation": results.get("affiliation") if results else None,
        "topics": results.get("topics") if results else None,
        "x_concepts": results.get("x_concepts") if results else None,
        "generated_profile": response_text if results else None
    }])], ignore_index=True)

display(df_member_info)



--- Profil für Gregor Anderluh ---
Gregor Anderluh is an expert in lipid membrane structure and behavior, with a strong focus on its role in biological systems across cell biology and biophysics. His research integrates concepts from biochemistry, genetics, and chemistry to advance our understanding of how these structures function within living organisms.

--- Profil für Nicholas Arndt ---
Institut des Sciences de la Terre is where Nicholas Arndt conducts his research. His work primarily involves the analysis of geological samples using advanced analytical techniques to identify and study Earth's dynamic processes.

--- Profil für Yuling Bai ---
Yuling Bai specializes in the study of plant-microbe interactions and plant-immune systems, employing a multidisciplinary approach that integrates biology, genetics, chemistry, and biochemistry to advance our understanding of how microbes shape plant health and resilience. Their research is at the forefront of theoretical and applied studies

,name,affiliation,topics,x_concepts,generated_profile
0,Gregor Anderluh,National Institute of Chemistry,[Lipid Membrane Structure and Behavior],"[Biology, Chemistry, Biochemistry, Genetics]",Gregor Anderluh is an expert in lipid membrane...
1,Nicholas Arndt,Institut des Sciences de la Terre,[Geological and Geochemical Analysis],[],Institut des Sciences de la Terre is where Nic...
2,Yuling Bai,Wageningen University & Research,[Plant-Microbe Interactions and Immunity],"[Biology, Genetics, Chemistry, Biochemistry]",Yuling Bai specializes in the study of plant-m...
3,Michael Benton,University of Bristol,[Evolution and Paleontology Studies],[],Michael Benton is a researcher at the Universi...
4,Petra Boevink,James Hutton Institute,[Plant-Microbe Interactions and Immunity],"[Biology, Chemistry, Genetics, Biochemistry]",Petra Boevink is a researcher at the James Hut...
...,...,...,...,...,...
63,Yusak Susilo,BOKU University,[Urban Transport and Accessibility],[],Yusak Susilo is a distinguished researcher at ...
64,Pilar Villamor,GNS Science,[earthquake and tectonic studies],"[Geology, Seismology]",Pilar Villamor specializes in understanding th...
65,Kirsi Virrantaus,Aalto University,[Geographic Information Systems Studies],[],Her research focuses on geographic information...
66,Xiaogang Wu,New York University Shanghai,[China's Socioeconomic Reforms and Governance],[],Xiaogang Wu has conducted extensive research o...


In [ ]:
from ddgs import DDGS
import requests
from bs4 import BeautifulSoup

# Suche durchführen
query = "Prof Kim A. Boutilier"
results = DDGS().text(query, region='wt-wt', max_results=5)

# Ergebnisse anzeigen
for r in results:
    print(f"\n{r['title']}\n{r['href']}")

# Textinhalt der ersten Seite scrapen
url = list(results)[0]["href"]
html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}).text
soup = BeautifulSoup(html, "html.parser")

text = ' '.join(p.get_text() for p in soup.find_all("p"))
print("\n--- Textauszug ---\n", text[:1000])



dr. KA (Kim) Boutilier - WUR
https://www.wur.nl/en/persons/kim-boutilier.htm

Kim Boutilier's research works | Wageningen University & Research ...
https://www.researchgate.net/scientific-contributions/Kim-Boutilier-20800113

Kim Boutilier - AIChE
https://www.aiche.org/sbe/community/bio/kim-boutilier

Kim Boutilier - Wageningen University & Research
https://research.wur.nl/en/persons/kim-boutilier

Kim Boutilier Inventions, Patents and Patent Applications - Justia ...
https://patents.justia.com/inventor/kim-boutilier

--- Textauszug ---
 Our Research Our research is focussed on understanding the molecular-genetic basis for plant regeneration through embryogenesis. We study different types of in vitro regeneration, including embryogenesis from haploid male and female gametophytes (pollen and embryo sac) and embryogenesis from vegetative tissues (somatic embryogenesis), as well as different types of seed-based embryogenesis systems (zygotic embryogenesis, egg cell parthenogenesis, mater

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Kim A. Boutilier")


"We found 10 Kimberley Boutilier's profiles > Get contact information, phone numbers, home addresses, age, background check, photos, and other public records [Updated: May 7, 2025]. Hi, my name is Kathleen Tracey, the couple in the picture are my long time friend Kim (Van… Kathleen Tracey needs your support for Gerard's battle with Cancer Search Boutilier family obituaries and memoriams on Legacy.com. There are 326 obituaries and memoriams for the surname Boutilier . View Profile 370 Jeremy Ln, Weatherford, TX 76088 also known as Kim A Witt Kimberlya Witt Kimberly A Witt Kim A Boutilier Kimberly A Boutilier Kimberley A Boutilier related to Carolyn Richardson, 66 James Dunn, 56 Matthew Fisher Barbara Boutilier , 80 Hazel Boutilier , 57 has lived in Weatherford, TX Kenosha, WI phone number (817) 475-6213 (817 ... 370 Jeremy Ln, Weatherford, TX 76088 Verified also known as Kimberly A Witt Kimberlya Witt Kim A Witt Kimberly A Boutilier Kim A Boutilier Kimberley A Boutilier related to Carol

In [ ]:


from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

search.invoke("Gregor Anderluh")

'September 11, 2025 - Schulz. Boutilier went to Vancouver Film School in 1996, as well as the University of King\'s College. 3 weeks ago - The EP featured appearances from Pusha T and Kurtis Blow on the songs "Strangers" and "Going Diamond" respectively, as well as song writing credits from the likes of Ryan Tedder (of OneRepublic), Grammy award-winning songwriter Evan Bogart, Cherry Cherry Boom Boom, and producer Doctor Rosen Rosen. The EP received mixed reviews, comparing Boutilier to the likes of Katy Perry, Nicki Minaj, and Gwen Stefani. July 11, 2025 - Paul André Boutilier (born May 3, 1963) is a Canadian former professional ice hockey defenceman who played with several National Hockey League teams in the 1980s. September 22, 2025 - Teahoon Park; Chihyun Park; Byeonggwan Kim; Haejin Shin; Eunkyoung Kim (2013). "Flexible PEDOT electrodes with large thermoelectric power factors to generate electricity by the touch of fingertips". Energy & Environmental Science. 6 (3): 788. doi:10.10

In [ ]:
search = DuckDuckGoSearchResults(output_format="list", num_results=5)

search.invoke("Prof Gregor Anderluh")

[{'snippet': 'Es ist bundesweit nicht einheitlich geregelt, welchen Titel ein Juniorprofessor führen soll (Jun.-Prof. oder Prof.) und ob ihm das Promotionsrecht zuerkannt wird.',
  'title': 'Professor – Wikipedia',
  'link': 'https://de.wikipedia.org/wiki/Professor'},
 {'snippet': 'Feb 3, 2025 · Der Titel des Professors beziehungsweise der Professorin (kurz „Prof.“ beziehungsweise „Prof.in“) ist der höchste akademische Titel, der in der Wissenschaft verliehen …',
  'title': 'Professorentitel: Welche gibt es , wer darf sie führen? | academics',
  'link': 'https://www.academics.de/ratgeber/professoren-titel-titelfuehrung'},
 {'snippet': 'höchster akademischer Titel (der einem/einer [habilitierten] Hochschullehrer [in], verdienten Wissenschaftler [in], Künstler [in] o. Ä. verliehen wird) Grammatik ohne Plural Abkürzung Prof. …',
  'title': 'Professor Rechtschreibung, Bedeutung, Definition, Herkunft Duden',
  'link': 'https://www.duden.de/rechtschreibung/Professor'},
 {'snippet': 'Um dies 

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region="de-de")

search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format="list", num_results=5)

search.invoke("Prof Gregor Anderluh")

[{'snippet': 'January 30, 2025 - He taught at the University of Navarra in Pamplona (Spain), University of Vermont, and McGill University as a visiting professor.',
  'title': 'Wikipedia Greg N. Gregoriou - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Greg_N._Gregoriou'},
 {'snippet': 'September 20, 2025 - In 1960, he obtained employment as a philosophy instructor at Washington College, and in 1961 he received his doctorate at Columbia as an Irwin Edman Scholar and with Distinction in History after his dissertation on Giovanni Gentile. Gregor became assistant professor of philosophy at the University of Hawaii from 1961 to 1964.',
  'title': 'Wikipedia A. James Gregor - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/A._James_Gregor'},
 {'snippet': 'April 22, 2025 - Georges Anderla (March 27, 1921 in Prague - April 26, 2005 in Antibes) was a Czech-born French economist . While working for the OECD in 1973, he created a statistical model of the accumulation of human knowled

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults

# 🧭 Richtige Region und Sprache
wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", safesearch="strict")

search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format="list", num_results=3)

# 🗝️ Suchbegriff präziser formulieren
query = "Kim A. Boutilier site:researchgate.net"

results = search.invoke(query)

for i, r in enumerate(results, start=1):
    print(f"🔹 Ergebnis {i}:")
    print(f"  Titel: {r['title']}")
    print(f"  Link: {r['link']}")
    print(f"  Beschreibung: {r['snippet']}\n")


🔹 Ergebnis 1:
  Titel: (PDF) Ectopic expression of BABY BOOM triggers a conversion from...
  Link: https://www.researchgate.net/publication/11215105_Ectopic_expression_of_BABY_BOOM_triggers_a_conversion_from_vegetative_to_embryonic_growth
  Beschreibung: from Vegetative to Embryonic Growth. Kim Boutilier , a,b,1.To whom correspondence should be addressed. E-mail k. a . boutilier @. plant.wag-ur.nl; fax 31-317-42-31-10.

🔹 Ergebnis 2:
  Titel: (PDF) Plasticity in Cell Division Patterns and Auxin Transport...
  Link: https://www.researchgate.net/publication/263273536_Plasticity_in_Cell_Division_Patterns_and_Auxin_Transport_Dependency_during_in_Vitro_Embryogenesis_in_Brassica_napus
  Beschreibung: Remko Offringa, e. and Kim Boutilier . a,1. a. Plant Research International, 6700 AP Wageningen, The Netherlands.

🔹 Ergebnis 3:
  Titel: (PDF) Auxin biosythesis maintains embryo identity and growth during...
  Link: https://www.researchgate.net/publication/356791425_Auxin_biosythesis_maintains_

In [11]:
import requests
from pprint import pprint

def fetch_openalex_id(name: str) -> dict | None:
    search_name = name.replace(" ", "+")
    base_url = f"https://api.openalex.org/authors?filter=display_name.search:{search_name}"
    response = requests.get(base_url)

    if response.status_code == 200:
        data = response.json()
        results_list = data.get('results', [])
        
        if not results_list:
            #st.warning(f"⚠️ Keine Ergebnisse für {name}")
            return None

        first_result = results_list[0]
        topics = first_result.get('topics', [])
        sorted_topics = sorted(topics, key=lambda x: x.get('count', 0), reverse=True)
        top_topics = [t['display_name'] for t in sorted_topics[:3]]

        affiliations = first_result.get('affiliations', [])
        
        # Funktion, um die aktuellste Institution zu finden
        def get_most_recent_affiliation(affiliations):
            if not affiliations:
                return None
            # Sortiere nach dem höchsten Jahr in der `years`-Liste
            most_recent = max(affiliations, key=lambda x: max(x.get('years', [0])))
            return most_recent['institution']

        # Aktuellste Institution abrufen
        most_recent_affiliation = get_most_recent_affiliation(affiliations)
        x_concepts = first_result.get('x_concepts', [])
        x_concepts_fields_list = [
            concept['display_name'] for concept in x_concepts if concept.get("score", 0) >= 90
        ]

        return {
            "Name": name,
            "topics": top_topics,
            "affiliation": most_recent_affiliation['display_name'] if most_recent_affiliation else None,
            "x_concepts": x_concepts_fields_list
        }

    else:
        #st.error(f"⚠️ Fehler bei Anfrage: {response.status_code}")
        return None
    
fetch_openalex_id("Gregor Anderluh")

{'Name': 'Gregor Anderluh',
 'topics': ['Lipid Membrane Structure and Behavior',
  'Marine Invertebrate Physiology and Ecology',
  'RNA and protein synthesis mechanisms'],
 'affiliation': 'National Institute of Chemistry',
 'x_concepts': []}